## "Анализ веб-документов" Unsupervised подход

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

## Обработка данных

Обкачивал и обрабатывал данные я во время подготовки проекта, как я это делал можно посмотреть [здесь](https://github.com/SherAndrei/sphere_ml_spring_2021/blob/master/project/Project.ipynb). Обкаченные данные лежат на каггле по [этой ссылке](https://www.kaggle.com/sherandrei/spheredata).

In [2]:
path_to_data = "../input/anomaly-detection-competition-ml1-ts-spring-2021/"

traindf = pd.read_csv(path_to_data + 'train_groups.csv', sep=',', index_col='pair_id')
testdf = pd.read_csv(path_to_data + 'test_groups.csv', index_col='pair_id')

In [3]:
filesdf = pd.read_csv("../input/spheredata/processed_data/data.csv", index_col='doc_id', usecols=['doc_id', 'processed_title', 'auxiliary', 'text'], na_filter=False)

## Параллелизм

Вспомогательные функции для быстрой обработки данных

In [4]:
from functools import partial
from tqdm.contrib.concurrent import process_map

In [5]:
def apply_to_array(arr, func, **kwargs):
    func_wrapper = partial(func, **kwargs)
    return process_map(func_wrapper, arr, chunksize=1, max_workers=4)

In [6]:
def apply_to_subdf(df, attr, func, **kwargs):
    values = getattr(df, attr).unique()
    splitted_dfs = [df[getattr(df, attr) == val] for val in values]
    return apply_to_array(splitted_dfs, func, **kwargs)

## Нахождение признаков

In [7]:
import numpy as np
from scipy import sparse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn import metrics

Вытащим множество из заголовков, ключевых слов и описания в колонку head, 10 важных слов из body в одноименную колонку

In [8]:
def top_words(dataset, n):
    vec = TfidfVectorizer(max_features=n, token_pattern='[а-яА-Яa-zA-Z]{4,30}', stop_words={'english'})
    vec.fit(dataset)
    return vec.get_feature_names()

def important_words(doc_df):
    data = doc_df.to_dict('records')[0]

    head_set = set((data['auxiliary'] + ' ' + data['processed_title']).split())
    
    text_len = len(data['text'])
    body_set = set()
    if text_len > 0:
        top = top_words([data['text']], min(10, text_len))
        body_set = set(top)

    return " ".join(head_set), " ".join(body_set)

In [9]:
for i, (head, body) in enumerate(apply_to_subdf(filesdf, 'index', important_words), start=1):
    filesdf.at[i, 'head_words'] = head
    filesdf.at[i, 'body_words'] = body

filesdf.head(4)

  0%|          | 0/28026 [00:00<?, ?it/s]

,processed_title,auxiliary,text,head_words,body_words
doc_id,,,,,
1,аншина центр репродукция генетика фертимед,аншина центр репродукция генетика фертимед док...,аншина центр репродукция генетика фертимед ска...,рационализация размер бесплодие пациент привод...,анализ кровь уровень определение лабораторный ...
2,перевод киви кошелёк,,перевод киви кошелёк перевод киви кошелёк пере...,киви перевод кошелёк,кошел карта деньга перевести электронный карто...
3,проект патруль реабилитация духовный существо ...,,проект патруль реабилитация духовный существо ...,деньга финансы реабилитация проект существо те...,являться деньга экономический денежный товар с...
4,блог клуб преподавание начальный класс портал ...,блог клуб преподавание начальный класс,размер шрифт цвет сайт изображение выклый шриф...,профессиональный начальный портал блог чеченск...,коллектив урок учитель труд обучение деятельно...


In [10]:
def word_occurences_distance_matrix(dataset):
    vec = CountVectorizer(binary=True, token_pattern='[а-яА-Яa-zA-Z]{3,30}', max_df=300, stop_words={'english'})
    vectors = vec.fit_transform(dataset)
    return (vectors @ vectors.T).A

def cosine_distance_matrix(dataset):
    vectorizer = TfidfVectorizer(token_pattern='[а-яА-Яa-zA-Z]{3,30}', min_df=5, max_df=300, ngram_range=(1, 2), stop_words={'english'})
    matrix = vectorizer.fit_transform(dataset)
    return metrics.pairwise.cosine_distances(matrix)

In [11]:
def feature_vectors(dataset, alpha=1.):
    cos_matrix = cosine_distance_matrix(dataset)
    sorted_cos_matrix = np.sort(cos_matrix)[:, 1:]

    word_matrix = word_occurences_distance_matrix(dataset)
    sorted_word_matrix = (-np.sort(-word_matrix))[:, 1:]

    out_vectors = sorted_cos_matrix + alpha / sorted_word_matrix
    return out_vectors

def get_feature_vectors(dataset, n_features, alpha):
    cur_n_features = min(n_features, len(dataset))
    return feature_vectors(dataset, alpha)[:, :cur_n_features]

def extract_features(group_df, n_features, alpha):
    datadf = filesdf.loc[group_df['doc_id'].values]
    
    X = get_feature_vectors(datadf.head_words, n_features, alpha)
    X += get_feature_vectors(datadf.body_words, n_features, alpha)
    
    return group_df.group_id.values[0], np.nan_to_num(X, nan=3)

def extract_features_from_dataframe(df, kwargs):
    return {group_id: features for group_id, features in apply_to_subdf(df, 'group_id', extract_features, **kwargs)}

## Подбор параметров

In [12]:
from sklearn.cluster import DBSCAN
from sklearn.model_selection import ParameterGrid
from operator import itemgetter

In [13]:
y_train = traindf['target'].to_numpy()

In [14]:
def convert_to_target(labels):
    return np.array([0 if label == -1 else 1 for label in labels])

In [15]:
def one_group_score(params, input_, y):
    X = input_
    labels = DBSCAN(**params).fit(X).labels_
    return params, metrics.f1_score(convert_to_target(labels), y)

def several_groups_score(params, input_, y):
    X = input_
    all_labels = []
    for group_id in X:
        group_features = X[group_id]
        all_labels.extend(DBSCAN(**params).fit(group_features).labels_)
    return params, metrics.f1_score(convert_to_target(all_labels), y)

In [16]:
def split_into_features_and_attrs(params):
    features = {p[3:]: params[p] for p in params if p.startswith("f__")}
    attrs = {p: params[p] for p in params if not p.startswith("f__")}
    return features, attrs

def one_group_score_w_features(params, input_, y):
    df = input_
    features, attrs = split_into_features_and_attrs(params)

    _, X = extract_features(df, **features)
    _, score = one_group_score(attrs, X, y)
    return params, score

def several_groups_score_w_features(params, input_, y):
    df = input_
    features, attrs = split_into_features_and_attrs(params)

    X = extract_features_from_dataframe(df, features)
    _, score = several_groups_score(attrs, X, y)
    return params, score

In [17]:
def GridSearch(input_, y_true, grid, func_to_search_with):
    params_to_mean_score = apply_to_array(ParameterGrid(grid), func_to_search_with, input_=input_, y=y_true)
    return params_to_mean_score, max(params_to_mean_score, key=itemgetter(1))

In [18]:
grid = {
    'f__alpha': [3],
    'f__n_features': range(20, 61, 10),
    'eps': np.logspace(-4, 0, 10),
    'min_samples': range(5, 16, 2),
    'metric': ['cosine']
}

In [19]:
all_data, best = GridSearch(traindf, y_train, grid, several_groups_score_w_features)

  0%|          | 0/300 [00:00<?, ?it/s]

### Результаты валидации

In [20]:
best

({'eps': 0.016681005372000592,
  'f__alpha': 3,
  'f__n_features': 40,
  'metric': 'cosine',
  'min_samples': 15},
 0.6483616111902603)

In [21]:
sorted(all_data, key=itemgetter(1), reverse=True)[:5]

[({'eps': 0.016681005372000592,
   'f__alpha': 3,
   'f__n_features': 40,
   'metric': 'cosine',
   'min_samples': 15},
  0.6483616111902603),
 ({'eps': 0.046415888336127774,
   'f__alpha': 3,
   'f__n_features': 40,
   'metric': 'cosine',
   'min_samples': 15},
  0.6483616111902603),
 ({'eps': 0.12915496650148828,
   'f__alpha': 3,
   'f__n_features': 40,
   'metric': 'cosine',
   'min_samples': 15},
  0.6483616111902603),
 ({'eps': 0.3593813663804626,
   'f__alpha': 3,
   'f__n_features': 40,
   'metric': 'cosine',
   'min_samples': 15},
  0.6483616111902603),
 ({'eps': 0.016681005372000592,
   'f__alpha': 3,
   'f__n_features': 40,
   'metric': 'cosine',
   'min_samples': 13},
  0.6477052359405301)]

In [22]:
best_features, best_attrs = split_into_features_and_attrs(best[0])

In [23]:
best_features, best_attrs

({'alpha': 3, 'n_features': 40},
 {'eps': 0.016681005372000592, 'metric': 'cosine', 'min_samples': 15})

### Считываем test и делаем predict

In [24]:
def predict(best_params, X):
    all_labels = []
    for group_id in X:
        group_features = X[group_id]
        all_labels.extend(DBSCAN(**best_params).fit(group_features).labels_)
    return convert_to_target(all_labels)

In [25]:
X_test = extract_features_from_dataframe(testdf, best_features)

  0%|          | 0/180 [00:00<?, ?it/s]

In [26]:
testdf['target'] = predict(best_attrs, X_test)

testdf.drop(columns=['group_id', 'doc_id']).to_csv('dbcan_solution.csv')

Итоговый скор 0.60410